In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## packages for llama 3b 
!pip install -q transformers==4.44.2 peft==0.11.1 accelerate bitsandbytes==0.43.1 datasets==2.21.0

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
LORA_PATH = "/kaggle/input/3btesting/llama3_price_model"  

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base_model, LORA_PATH)

2025-10-13 08:51:48.534215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760345508.789372     166 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760345508.861412     166 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import login

login(token="hf_xxxxxxx")

In [ ]:
!pip install -U peft

In [ ]:
# 1) Uninstall bitsandbytes and triton (and optionally reinstall peft cleanly)
!pip uninstall -y bitsandbytes triton || true
!pip install -q --upgrade peft transformers accelerate

In [ ]:
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer
from peft import PeftModel
from transformers import AutoModelForCausalLM

# Paths (adjust if needed)
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
LORA_PATH = "/kaggle/input/3btesting/llama3_price_model"
TEST_FILE = "/kaggle/input/3btesting/testCleaned.csv"

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(base_model, LORA_PATH)
model.eval()

# Load a few test samples
df = pd.read_csv(TEST_FILE)
df_test = df.head(5)  # small batch to test

# Tokenize and move to device
inputs = tokenizer(
    df_test['catalog_content'].tolist(),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
).to(device)

import re

# Generate predictions
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=10
    )

# Decode predictions
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Extract numeric values from the output
decoded_prices = []
for text in decoded:
    match = re.search(r"\d+\.?\d*", text)  # find first number (integer or decimal)
    if match:
        decoded_prices.append(float(match.group()))
    else:
        decoded_prices.append(None)

# Show test results
for sid, price in zip(df_test['sample_id'], decoded_prices):
    print(f"Sample ID: {sid}, Predicted Price: {price}")



In [ ]:
import re

# Generate predictions
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=10
    )

# Decode predictions
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Extract numeric values and round to 2 decimals
decoded_prices = []
for text in decoded:
    match = re.search(r"\d+\.?\d*", text)  # find first number
    if match:
        decoded_prices.append(round(float(match.group()), 2))
    else:
        decoded_prices.append(None)

# Show test results
for sid, price in zip(df_test['sample_id'], decoded_prices):
    print(f"Sample ID: {sid}, Predicted Price: {price}")


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Paths
TEST_FILE = "/kaggle/input/3btesting/testCleaned.csv"  # your test CSV with sample_id and catalog_content
OUTPUT_FILE = "predictions.csv"

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer & base model
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
LORA_PATH = "/kaggle/input/3btesting/llama3_price_model"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, LORA_PATH)
model.eval()
model.to(device)

# Load test data
df = pd.read_csv(TEST_FILE)

BATCH_SIZE = 32
predictions = []

# Batch-wise prediction
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Predicting prices"):
    batch = df.iloc[i:i+BATCH_SIZE]

    # Add prompt to encourage 2-decimal output
    prompts = [text + "\nPredict the price in USD with up to 2 decimal points:" for text in batch['catalog_content']]

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract numeric price and format to 2 decimals
    for sid, text in zip(batch['sample_id'], decoded):
        match = re.search(r"\d+\.?\d*", text)
        price = f"{float(match.group()):.2f}" if match else None
        predictions.append({"sample_id": sid, "predicted_price": price})

# Save all predictions
pred_df = pd.DataFrame(predictions)
pred_df.to_csv(OUTPUT_FILE, index=False)
print(f"Predictions saved to {OUTPUT_FILE}")


In [7]:
import pandas as pd
import torch
from tqdm import tqdm
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Paths
TEST_FILE = "/kaggle/input/3btesting/testCleaned.csv"  # your test CSV
OUTPUT_FILE = "predictions_25k.csv"

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer & base model
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
LORA_PATH = "/kaggle/input/3btesting/llama3_price_model"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, LORA_PATH)
model.eval()
model.to(device)

# Load test data (limit to first 5k) 
df = pd.read_csv(TEST_FILE, skiprows=range(1, 55001), nrows=10000) 

BATCH_SIZE = 32
predictions = []

# Batch-wise prediction
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Predicting prices (first 25k)"):
    batch = df.iloc[i:i+BATCH_SIZE]

    # Add prompt to encourage 2-decimal output
    prompts = [text + "\nPredict the price in USD with up to 2 decimal points:" for text in batch['catalog_content']]

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract numeric price and format to 2 decimals
    for sid, text in zip(batch['sample_id'], decoded):
        match = re.search(r"\d+\.?\d*", text)
        price = f"{float(match.group()):.2f}" if match else None
        predictions.append({"sample_id": sid, "predicted_price": price})

# Save all predictions
pred_df = pd.DataFrame(predictions)
pred_df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Predictions for second 5k samples saved to {OUTPUT_FILE}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Predicting prices (first 25k): 100%|██████████| 313/313 [42:11<00:00,  8.09s/it]

✅ Predictions for second 5k samples saved to predictions_25k.csv


In [8]:
import pandas as pd
import torch
from tqdm import tqdm
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Paths
TEST_FILE = "/kaggle/input/3btesting/testCleaned.csv"  # your test CSV
OUTPUT_FILE = "predictions_25k.csv"

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer & base model
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
LORA_PATH = "/kaggle/input/3btesting/llama3_price_model"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, LORA_PATH)
model.eval()
model.to(device)

# Load test data (limit to first 5k) 
df = pd.read_csv(TEST_FILE, skiprows=range(1, 65001), nrows=10000) 

BATCH_SIZE = 32
predictions = []

# Batch-wise prediction
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Predicting prices (first 25k)"):
    batch = df.iloc[i:i+BATCH_SIZE]

    # Add prompt to encourage 2-decimal output
    prompts = [text + "\nPredict the price in USD with up to 2 decimal points:" for text in batch['catalog_content']]

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract numeric price and format to 2 decimals
    for sid, text in zip(batch['sample_id'], decoded):
        match = re.search(r"\d+\.?\d*", text)
        price = f"{float(match.group()):.2f}" if match else None
        predictions.append({"sample_id": sid, "predicted_price": price})

# Save all predictions
pred_df = pd.DataFrame(predictions)
pred_df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Predictions for second 5k samples saved to {OUTPUT_FILE}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Predicting prices (first 25k): 100%|██████████| 313/313 [42:57<00:00,  8.24s/it]

✅ Predictions for second 5k samples saved to predictions_25k.csv


In [ ]:
#testing is done in batches of 5k and 10k 
#all batches not visible